In [1]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import json
import itertools
from statsmodels.stats.inter_rater import fleiss_kappa

In [2]:
# Define a helper function to load annotations from jsonl files
def load_annotations(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        annotations = [json.loads(line) for line in f.readlines()]
    return annotations

# Load the data for each annotator
file_1 = load_annotations('iteracja2/knoculak.jsonl')
file_2 = load_annotations('iteracja2/jtarasenko.jsonl')
file_3 = load_annotations('iteracja2/bwrzalski.jsonl')

In [3]:
file_1 = file_1[50:100]
file_2 = file_2[50:100]
file_3 = file_3[50:100]

In [4]:
# Define the categories we are working with (as mentioned in the prompt)
categories = ["Jedzenie", "Sen", "Widoki", "Obsługa", "None"]

In [5]:
# Function to extract and prepare annotations for Kappa calculation
def prepare_annotations_for_kappa(annotations, categories):
    # Create a list of lists where each sublist contains labels for each text sample
    all_labels = []
    for annotation in annotations:
        labels = ['None'] * len(annotation['text'])  # Assuming length of the text as the reference
        for label in annotation['label']:
            start, end, category = label
            for i in range(start, end):
                labels[i] = category
        all_labels.append(labels)
    return all_labels

In [6]:
# Prepare the annotations for each annotator
annotations_1 = prepare_annotations_for_kappa(file_1, categories)
annotations_2 = prepare_annotations_for_kappa(file_2, categories)
annotations_3 = prepare_annotations_for_kappa(file_3, categories)

# Flatten the annotations (as we are considering all the labeled fragments and ignoring non-annotated)
flat_annotations_1 = list(itertools.chain(*annotations_1))
flat_annotations_2 = list(itertools.chain(*annotations_2))
flat_annotations_3 = list(itertools.chain(*annotations_3))

In [7]:
def filtered_annotations_for_cohen (ann_1, ann_2):
    filtered_annotations_1_for_cohen = []
    filtered_annotations_2_for_cohen = []

    for a1, a2 in zip(ann_1, ann_2):
        if a1 != "None" or a2 != "None":
            filtered_annotations_1_for_cohen.append(a1)
            filtered_annotations_2_for_cohen.append(a2)

    return filtered_annotations_1_for_cohen, filtered_annotations_2_for_cohen

In [8]:
filtered_annotations_11_for_cohen, filtered_annotations_12_for_cohen = filtered_annotations_for_cohen (flat_annotations_1, flat_annotations_2)
filtered_annotations_21_for_cohen, filtered_annotations_22_for_cohen = filtered_annotations_for_cohen (flat_annotations_2, flat_annotations_3)
filtered_annotations_31_for_cohen, filtered_annotations_32_for_cohen = filtered_annotations_for_cohen (flat_annotations_1, flat_annotations_3)

In [9]:
print(len(flat_annotations_1))
print(len(filtered_annotations_11_for_cohen))
print(len(filtered_annotations_21_for_cohen))
print(len(filtered_annotations_31_for_cohen))

36059
9660
8728
7848


In [10]:
# Calculate Cohen's Kappa for two annotators (jtarasenko and knoculak)
kappa_cohen1 = cohen_kappa_score(filtered_annotations_11_for_cohen, filtered_annotations_12_for_cohen, labels=categories)
kappa_cohen2 = cohen_kappa_score(filtered_annotations_21_for_cohen, filtered_annotations_22_for_cohen, labels=categories)
kappa_cohen3 = cohen_kappa_score(filtered_annotations_31_for_cohen, filtered_annotations_32_for_cohen, labels=categories)

print(kappa_cohen1, kappa_cohen2, kappa_cohen3)

0.3746714962058918 0.31646933657347687 0.2668656142679201


In [11]:
def filtered_annotations_for_fleissa (ann_1, ann_2, ann_3):
    filtered_annotations_1 = []
    filtered_annotations_2 = []
    filtered_annotations_3 = []

    for a1, a2, a3 in zip(ann_1, ann_2, ann_3):
        if a1 != "None" or a2 != "None" or a3 != "None":
            filtered_annotations_1.append(a1)
            filtered_annotations_2.append(a2)
            filtered_annotations_3.append(a3)

    return filtered_annotations_1, filtered_annotations_2, filtered_annotations_3

filtered_annotations_1, filtered_annotations_2, filtered_annotations_3 = filtered_annotations_for_fleissa(flat_annotations_1, flat_annotations_2, flat_annotations_3)

print(filtered_annotations_1)
print(filtered_annotations_2)
print(filtered_annotations_3)


['Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa', 'Obsługa'

In [12]:
print(len(flat_annotations_1))
print(len(filtered_annotations_1))

36059
9859


In [13]:
data = []
for a1, a2, a3 in zip(filtered_annotations_1, filtered_annotations_2, filtered_annotations_3):
    label_count = [0] * len(categories)
    label_count[categories.index(a1)] += 1
    label_count[categories.index(a2)] += 1
    label_count[categories.index(a3)] += 1
    data.append(label_count)

data

[[0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 2, 1],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0, 1, 2],
 [0, 0, 0,

In [14]:
# Calculate Fleiss' Kappa for 3 annotators
kappa_fleiss = fleiss_kappa(data)
print(kappa_fleiss)

0.37904222663074943
